In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Specialist Classification - Specialist Classification.csv')

In [3]:
df_train = df.iloc[:, [3,5]]

In [4]:
df_train.head()

,Problem In English,Specialist
0,I have a lot of headache for three days,Medicine
1,I have to hold my head as soon as I am in the ...,Medicine
2,My sister's eyes and headache for a few days,Medicine
3,My mom causes a lot of pain on the two sides o...,Medicine
4,I have a lot of headache,Medicine


In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

In [6]:
df_train.iloc[:,1]

0             Medicine
1             Medicine
2             Medicine
3             Medicine
4             Medicine
            ...       
653            Surgery
654            Dentist
655               Skin
656            Surgery
657    Ophthalmologist
Name: Specialist, Length: 658, dtype: object

In [7]:
target_tokenizer = Tokenizer(30)
target_tokenizer.fit_on_texts(df_train.iloc[:,1])
target_vocab_size = len(target_tokenizer.word_index) + 1
df_train.iloc[:,1] = np.array(target_tokenizer.texts_to_sequences(df_train.iloc[:,1])) -1
# df_train.iloc[:,1] = df_train.iloc[:,1].reshape(df_train.iloc[:,1].shape[0])
print("target_vocab_size:",target_vocab_size)

target_vocab_size: 10


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [8]:
target_tokenizer.word_index.items()

dict_items([('medicine', 1), ('ophthalmologist', 2), ('gynecologist', 3), ('surgery', 4), ('orthopedic', 5), ('cardiologist', 6), ('skin', 7), ('dentist', 8), ('ent', 9)])

In [9]:
df_train.head()

,Problem In English,Specialist
0,I have a lot of headache for three days,0
1,I have to hold my head as soon as I am in the ...,0
2,My sister's eyes and headache for a few days,0
3,My mom causes a lot of pain on the two sides o...,0
4,I have a lot of headache,0


In [10]:
target_reverse_word_index = {v: k for k, v in list(target_tokenizer.word_index.items())}
target_reverse_word_index

{1: 'medicine',
 2: 'ophthalmologist',
 3: 'gynecologist',
 4: 'surgery',
 5: 'orthopedic',
 6: 'cardiologist',
 7: 'skin',
 8: 'dentist',
 9: 'ent'}

In [11]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(df_train.Specialist)
# y_test = to_categorical(df_test.Sentiment)

In [12]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.9 MB/s 
     |████████████████████████████████| 6.6 MB 66.3 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [14]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.iloc[:,0].tolist(),
    add_special_tokens=True,
    max_length=45,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [15]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [16]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [17]:
max_len = 45
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(9,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [18]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_split = 0.2,
  epochs=20,
    batch_size=36
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


15/15 [==============================] - 38s 576ms/step - loss: 1.7473 - balanced_accuracy: 0.4753 - val_loss: 2.6566 - val_balanced_accuracy: 0.0000e+00
Epoch 2/20
15/15 [==============================] - 5s 367ms/step - loss: 1.2830 - balanced_accuracy: 0.5875 - val_loss: 2.1099 - val_balanced_accuracy: 0.0985
Epoch 3/20
15/15 [==============================] - 6s 371ms/step - loss: 0.8508 - balanced_accuracy: 0.7395 - val_loss: 1.4530 - val_balanced_accuracy: 0.5455
Epoch 4/20
15/15 [==============================] - 6s 397ms/step - loss: 0.4998 - balanced_accuracy: 0.8707 - val_loss: 1.1065 - val_balanced_accuracy: 0.7197
Epoch 5/20
15/15 [==============================] - 6s 402ms/step - loss: 0.3455 - balanced_accuracy: 0.8916 - val_loss: 1.0825 - val_balanced_accuracy: 0.7197
Epoch 6/20
15/15 [==============================] - 6s 411ms/step - loss: 0.1768 - balanced_accuracy: 0.9563 - val_loss: 1.0263 - val_balanced_accuracy: 0.7424
Epoch 7/20
15/15 [============================

In [ ]:
# texts = input(str('I am having chest pain'))
texts = 'I had a bike accident yesterday, my knee is hurting'
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=45,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
prediction = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
# print(prediction[0].argsort()[::-1])
result= {}
for i in prediction[0].argsort()[::-1][:3]:
  result[target_reverse_word_index[i+1]] = prediction[0][i]



In [ ]:
result

In [ ]:
# print(prediction[0].argsort()[::-1])
for i in prediction[0].argsort()[::-1][:3]:
  print(target_reverse_word_index[i+1], prediction[0][i])

In [ ]:
target_reverse_word_index